# Cyclistic bike share analysis

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import plotly.express as px
#import plotly.graph_objects as go
import os
from datetime import date, timedelta
print("Completed import libraries!")

Completed import libraries!


## STEP 1: COLLECT DATA
 
Collect the previous 12 months of Cyclistic’s historical trip data. Download the Cyclistic trip data
[here](https://divvy-tripdata.s3.amazonaws.com/index.html).

The data has been made available by Motivate International Inc. under this [license](https://www.divvybikes.com/data-license-agreement).


In [3]:
## Read in data from my folder path: C:\Users\lucas\OneDrive\Desktop\VSCode\Cyclistic_bike_share\data
### create an empty list to store the dataframes
dfs = []
YM  = [202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303]
file_path = "C:/Users/lucas/OneDrive/Desktop/VSCode/Cyclistic_bike_share/data" 
### loop through the file names and read each file into a dataframe
for i in YM:
    filename = f"{file_path}/{i}-divvy-tripdata.csv"  # assuming the files are in CSV format
    df = pd.read_csv(filename)
    dfs.append(df)

### concatenate all dataframes in the list vertically
df_bike = pd.concat(dfs, axis=0)

### reset the index of the combined dataframe
df_bike = df_bike.reset_index(drop=True)

In [4]:
## Make a copy of the dataframe 
df = df_bike.copy()

## STEP 2: WRANGLE DATA

In [5]:
# See the last 6 rows of dataframe. Also use head() to get first 6 rows
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
5803715,BA8B9D1D0AAF5F25,electric_bike,2023-03-08 00:00:02,2023-03-08 00:06:25,Rush St & Hubbard St,KA1503000044,Canal St & Adams St,13011,41.890202,-87.626232,41.879255,-87.639904,member
5803716,841F8FE05B117E00,classic_bike,2023-03-21 14:24:42,2023-03-21 14:50:45,Sheridan Rd & Montrose Ave,TA1307000107,Sheridan Rd & Buena Ave,TA1309000027,41.961670,-87.654640,41.958494,-87.654966,member
5803717,25892EF6A0949B97,electric_bike,2023-03-21 16:48:52,2023-03-21 16:51:27,Halsted St & Roscoe St,TA1309000025,Broadway & Waveland Ave,13325,41.943681,-87.648830,41.949074,-87.648636,member
5803718,F6514CD31F2A9723,classic_bike,2023-03-16 20:36:05,2023-03-16 20:43:11,Sheridan Rd & Montrose Ave,TA1307000107,Broadway & Waveland Ave,13325,41.961670,-87.654640,41.949074,-87.648636,member
5803719,FEDA9EE828BC56BF,electric_bike,2023-03-27 19:58:08,2023-03-27 20:03:52,Clark St & Wellington Ave,TA1307000136,Broadway & Waveland Ave,13325,41.936490,-87.647581,41.949074,-87.648636,member


In [6]:
# Check the number of columns and rows in dataframe and see the list of columns and data types
print(df.shape)
df.info()

(5803720, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5803720 entries, 0 to 5803719
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 575.6+ MB


> Documention 23/6/2023 4:35 PM: 
>  *  5803720 columns and 13 rows. 
>  *  Some data type is not quite right such as: started_at, ended_at.

In [7]:
# See the User Type and Bike Type
print(df["member_casual"].unique())
print(df["rideable_type"].unique())

['member' 'casual']
['electric_bike' 'classic_bike' 'docked_bike']


### Decription of the datafrane
- 0   ride_id             Rental ID
- 1   rideable_type       Bike Type - ['electric_bike' 'classic_bike' 'docked_bike']
- 2   started_at          Local Start Time 
- 3   ended_at            Local End Time
- 4   start_station_name  Start Station Name
- 5   start_station_id    Start Station ID 
- 6   end_station_name    End Station Name
- 7   end_station_id      End Station ID 
- 8   start_lat           Start Latitude
- 9   start_lng           Start Longitude
- 10  end_lat             End Latitude
- 11  end_lng             End Longitude
- 12  member_casual       User Type - ['member' 'casual']

## STEP 3: CLEAN UP AND TRANSFORMING DATA TO PREPARE FOR ANALYSIS

There are a few problems we will need to fix:
1. The data type of the start time(started_at) and end time(ended_at) is object.
2. The data can only be aggregated at the ride-level, which is too granular. We will want to add some additional columns of data -- such as day, month, year -- that provide additional opportunities to aggregate the data.
3. We will need to add a calculated field for length of each ride (ride_length) for consistency.
4. Remove/fill "bad" data.


In [8]:
# Set index for ride_id

## Check if ride_id is unique and set ride_id to be index of the table
if df['ride_id'].is_unique == True:
    print("unique ride_id")
    df.set_index('ride_id', inplace= True)
    print("ride_id is the index now.")
else: print("ride_id is not unique")

unique ride_id
ride_id is the index now.


### Convert the data type 

In [9]:
## started_at, ended_at to datetime format YYYY-MM-DD hh:mm:ss
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

##  Changed the User Type into Casual_rider and Cyclistic_member 
### Casual riders: Customers who purchase single-ride or full-day passes 
### Cyclistic members: Customers who purchase annual memberships
df = df.replace({"member_casual" : { "member" : "Cyclistic_member", "casual" : "Casual_rider"}})
print(df["member_casual"].unique())

## Changed rideable_type, member_casual to categories, and rideable_type to string
df['rideable_type'] = df['rideable_type'].astype(str)
df['rideable_type'] = df['rideable_type'].astype('category')
df['member_casual'] = df['member_casual'].astype('category')

print("Completed Convert!")
print(df.dtypes)

['Cyclistic_member' 'Casual_rider']
Completed Convert!
rideable_type               category
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual               category
dtype: object


> Documention 23/6/2023 4:40 PM: 
>  *  Convert started_at, ended_at column to datetime format.
>  *  Replace values of member_casual column: { "member" : "Cyclistic_member", "casual" : "Casual_rider" }
>  *  Convert rideable_type and member_casual column to category.

### Created "ride_length" and "datetime" columns 
Calculated "ride_length" and Add columns year, month, day of week, day based on started_at column

In [10]:
## Add "ride_length" column (in seconds) calculation to dataframe
df["ride_length"] = df["ended_at"] - df["started_at"]
df["ride_length"] = df["ride_length"].dt.seconds

## extract year from started_at columnn, add it to new column year
df["year"] = df['started_at'].dt.year

## extract month from started_at columnn, add it to new column month
df["month"] = df['started_at'].dt.month

## extract day of week from started_at columnn, take 3 first character and add it to new column day_of_week
df["day_of_week"] = df['started_at'].dt.day_name().str[:3]

## extract day from started_at columnn, add it to new column day
df["day"] = df['started_at'].dt.day

> Documention 23/6/2023 4:45 PM: 
>  *  Calculate "ride_length" column(in seconds). 
>  *  Add columns "year", "month", "day_of_week", "day" based on "started_at".

In [11]:
# Check "ride_length" column is "clean" or not
df[df["ride_length"] <= 0].shape[0]

440

> Documentation 23/06/2023 4:46 PM:
>   * 440 negative values in "ride_length" column.
>   * There are some rides where "ride_length" shows up as negative. We will want to delete these rides.


In [12]:
# Remove negative values from ride_length column
df = df[df["ride_length"] > 0]

In [13]:
# # Check the number of columns and rows in dataframe, and see the last 6 rows
print(df.shape)
df.tail()

(5803280, 17)


,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,year,month,day_of_week,day
ride_id,,,,,,,,,,,,,,,,,
BA8B9D1D0AAF5F25,electric_bike,2023-03-08 00:00:02,2023-03-08 00:06:25,Rush St & Hubbard St,KA1503000044,Canal St & Adams St,13011,41.890202,-87.626232,41.879255,-87.639904,Cyclistic_member,383,2023,3,Wed,8
841F8FE05B117E00,classic_bike,2023-03-21 14:24:42,2023-03-21 14:50:45,Sheridan Rd & Montrose Ave,TA1307000107,Sheridan Rd & Buena Ave,TA1309000027,41.961670,-87.654640,41.958494,-87.654966,Cyclistic_member,1563,2023,3,Tue,21
25892EF6A0949B97,electric_bike,2023-03-21 16:48:52,2023-03-21 16:51:27,Halsted St & Roscoe St,TA1309000025,Broadway & Waveland Ave,13325,41.943681,-87.648830,41.949074,-87.648636,Cyclistic_member,155,2023,3,Tue,21
F6514CD31F2A9723,classic_bike,2023-03-16 20:36:05,2023-03-16 20:43:11,Sheridan Rd & Montrose Ave,TA1307000107,Broadway & Waveland Ave,13325,41.961670,-87.654640,41.949074,-87.648636,Cyclistic_member,426,2023,3,Thu,16
FEDA9EE828BC56BF,electric_bike,2023-03-27 19:58:08,2023-03-27 20:03:52,Clark St & Wellington Ave,TA1307000136,Broadway & Waveland Ave,13325,41.936490,-87.647581,41.949074,-87.648636,Cyclistic_member,344,2023,3,Mon,27


> Documentation 23/06/2023 4:48 PM 
>   * Remove the negative values( 440) in "ride_length" column 
>   * Data remaining : 5803280 rows and 17 columns.

### See the missing values in the dataframe

In [14]:
## Create missing_data function
def missing_data(df):
    null = (df.isnull().sum())/df.shape[0]*100  # Calculate the missing values percentage of the data for each column
    null = null.sort_values(ascending=False).to_frame() # Sort by missing values descending
    null.columns = ["Missing_data(%)"]  
    return null.T

## Calculate missing values
missing_data(df)

,end_station_id,end_station_name,start_station_id,start_station_name,end_lat,end_lng,rideable_type,ride_length,day_of_week,month,year,start_lng,member_casual,started_at,start_lat,ended_at,day
Missing_data(%),15.444249,15.441819,14.460977,14.458703,0.100891,0.100891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> Documention 23/06/2023 5:00 PM
> * Found missing data:   
>    -  end_station_id     : 15.444249%    
>    -  end_station_name   : 15.441819%  
>    -  start_station_id   : 14.460977%  
>    -  start_station_name : 14.458703%	     
>    -  end_lat            : 0.100891%   
>    -  end_lng            : 0.100891%	    
> * If the global address(start_lat, start_lng, end_lat, end_lng) is not missing then we don't need to remove missing data from end/start_station_id and end/start_station_name.

In [15]:
# See How many rows of end station information is all null?
missing_end_station = df[df["end_lat"].isnull() & df["end_lng"].isnull() & df["end_station_name"].isnull() & df["end_station_id"].isnull()]
missing_end_station.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,year,month,day_of_week,day
ride_id,,,,,,,,,,,,,,,,,
D21091A31145E144,docked_bike,2022-04-25 04:06:48,2022-04-26 05:06:49,Desplaines St & Randolph St,15535,NaN,NaN,41.884616,-87.644571,NaN,NaN,Casual_rider,3601,2022,4,Mon,25
E6C6B1D6FC42D248,docked_bike,2022-04-23 20:58:56,2022-04-27 05:00:32,Wabash Ave & Grand Ave,TA1307000117,NaN,NaN,41.891466,-87.626761,NaN,NaN,Casual_rider,28896,2022,4,Sat,23
F7FCC59039B672D7,classic_bike,2022-04-06 14:51:32,2022-04-07 15:51:23,Keystone Ave & Fullerton Ave,16010,NaN,NaN,41.924590,-87.727850,NaN,NaN,Casual_rider,3591,2022,4,Wed,6
1B9F07BED65435D9,docked_bike,2022-04-23 11:49:27,2022-04-25 04:56:57,Hoyne Ave & Balmoral Ave,655,NaN,NaN,41.979851,-87.681932,NaN,NaN,Casual_rider,61650,2022,4,Sat,23
300E45F342CC4647,classic_bike,2022-04-17 16:15:43,2022-04-18 17:15:35,Cicero Ave & Lake St,16903,NaN,NaN,41.886841,-87.745261,NaN,NaN,Casual_rider,3592,2022,4,Sun,17


### Remove missing end station values

In [16]:
# Remove rows missing all end station values
df = df[~((df["end_lat"].isnull())
          & (df["end_lng"].isnull())
          & (df["end_station_name"].isnull())
          & (df["end_station_id"].isnull())
          )]
print(f" Remove {missing_end_station.shape[0]} rows missing all end station values.")

 Remove 5855 missing end station values.


> Documention 23/06/2023 7:30 PM  
>   * Remove 5855 rows are missing all data of end station info.

### Create a start_end_station table contain start/end_station info 

Create start_end_station table so we can track the name or id of the station based on latitude and longitude values if needed.

In [17]:
# Create a table contain the name and id of all station 
station_info = df[["start_station_id","start_station_name","start_lat","start_lng"]]
station_info = station_info.drop_duplicates()
station_info = station_info.reset_index(drop=True)

# Rename the column of station table 
station_info = station_info.rename(columns={
    "start_station_id": "station_id",
    "start_station_name": "station_name",
    "start_lat": "lat",
    "start_lng": "lng"
})

In [11]:
# Fill missing values # We can use it if we need the data for the name or id of station

## Fill null values in end_station_id column with the id that have the same values end_station_name.
#station_info['station_id'] = station_info['station_id'].fillna(df['station_name'].map(End_id)) 

## STEP 4: CONDUCT DESCRIPTIVE ANALYSIS

We'll use Microsoft Power BI tool to calculate this step.

### Delete csv file **if needed.**

In [18]:
## Delete csv if needed use os.remove("file_name.csv")
#os.remove("bike_data.csv")
#os.remove("station_info.csv")

In [19]:
#EXPORT SUMMARY FILE FOR FURTHER ANALYSIS
## Create "bike_data.csv" file
df.to_csv("bike_data.csv", index=True)
print("The bike_data.csv file has been created.")

# Create csv file contain station info
station_info.to_csv("station_info.csv",index=False)
print("The station_info.csv file has been created.")

The bike_data.csv file has been created.
The station_info.csv file has been created.


## Documention from working with Microsoft Power BI

### Import data into PBI and Transform it.
● Add column name global_address for two table bike_data, station_info. Merge 2 columns latitude and longitude of the start station(separator: comma)  
● Remove columns lat, lng in table station_info.  
● Create relationships between station_info and bike_data through global_address column( many to many).  
● Create blank table to contain measure values: Description_analysis = #table({}, {}). Not working like I want.
### Conduct descriptive analysis.
● Calculate the mean of ride_length.    
● Calculate the max ride_length.    
● Calculate the mode of day_of_week.    
● Create measures percentage of Casual rides and percentage of Cyclistic members.   
● Add day_name column for better visualization.     
● Create measures for Top 1 day_name that user rent bicycles the most.   
### Visualize data, create a report to uncover insights.
### Presentation Cyclistic Bike Rentals Marketing Strategy.